In [1]:
include("../ToolBox/ToolBox.jl")
import JLD
import FileIO
using .ToolBox
using .Geometry
using Colors

In [2]:
function preprocess3_v1(product_folders, master_view, dem_path, subswath, result_folder, dem_nan,kernel = ones(4,14))
    
    n_coherence = length(product_folders)-1
    dem = 0
    lut1 = 0
    lut2 = 0
    lut_status = [false,false]
    precise_orbit = 0
    master_data = 0
    lines = 0
    samples = 0
    
    stride_line = floor(Int,size(kernel)[1]/2)
    stride_sample = floor(Int,size(kernel)[2]/2)
    
    # Save folder names
    path = joinpath(result_folder,"product_folders.txt")
    open(path, "w") do io
        for folder in product_folders
            write(io, folder * "\n")
        end
    end;
    
    # get pod paths
    pod_paths = [Load.pod_path(folder) for folder in product_folders]
    
    for polarization in ["VV","VH"]
        
        println(polarization)
        # get paths
        file_paths = [Load.slc_paths(folder, polarization, subswath) for folder in product_folders]
        meta = [Load.slc_meta(path[2]) for path in file_paths]
        calibration = [Load.slc_calibration(file_paths[i][3],meta[i]["t_0"]) 
                                    for i in 1:length(product_folders)];
        # load pod
        if precise_orbit == 0
            println("load POD")
            precise_orbit = [Load.precise_orbit(pod_paths[i],meta[i]["t_0"]) 
                                    for i in 1:length(product_folders)]
        end
        
        # load DEM
        if dem == 0
            println("load DEM")
            pad = 0.1
            dem = JLD.load(dem_path);
            footprint = SlcUtil.footprint(meta[2], master_view)
            latlon_window = ((minimum(footprint[1]), maximum(footprint[1])), (minimum(footprint[2]), maximum(footprint[2])))
            idx1 =(latlon_window[1][1] - pad) .<dem["lat"].< (latlon_window[1][2] + pad)
            idx2 =(latlon_window[2][1] - pad) .<dem["lon"].< (latlon_window[2][2] + pad)
            dem = (dem["lat"][idx1], dem["lon"][idx2], dem["height"][idx1,idx2]);
            dem[3][dem[3].== -32768] .= dem_nan; 
        end
        
        for i in 1:n_coherence
            println("i: ",i)
            if i == 1
                lut = lut1
                index = [2,1]
            else
                lut = lut2
                index = [2,3]
            end
            
            # coreg_slave data
            if !(lut_status[i])
                println("coreg no lut")
                slave_data,flat,lut =  coregister_slave(master_view,file_paths[index[2]][1],
                                                        meta[index],precise_orbit[index],dem)
                # save lut
                if i==1
                    lut1 = lut
                else
                    lut2 = lut
                end
                lut_status[i] = true
                lut_path = joinpath(result_folder,string(index[1])*string(index[2])*"_lut.jld")
                JLD.save(lut_path, "data", lut)
                
            else
                println("coreg with lut")
                slave_data,flat =  coregister_slave(master_view,file_paths[index[2]][1],
                                                        meta[index],precise_orbit[index],dem,lut)
            end
            
            slave_data,mosaic_view = SlcUtil.mosaic(slave_data,master_view,meta[index[1]]);
            flat,mosaic_view = SlcUtil.mosaic(flat,master_view,meta[index[1]]);
            slave_data = SlcUtil.calibrate_slave_data(slave_data, mosaic_view,lut1, calibration[index[2]]);
            
            if i == 1
            # Get master data
                master_data = Load.slc_data(file_paths[index[1]][1],master_view);
                master_data,mosaic_view = SlcUtil.mosaic(master_data,master_view,meta[index[1]]);
                master_data = SlcUtil.calibrate_data(master_data, Misc.flatten(mosaic_view...)..., calibration[index[1]]);
            end

            # Compute coherence
            complex_coherence, master_intensity, slave_intensity, lines, samples = SlcUtil.complex_coherence(
                                                                master_data, slave_data, flat, kernel, mosaic_view);

            slave_data = 0

            # subsample 
            complex_coherence = complex_coherence[1:stride_line:end,1:stride_sample:end]
            slave_intensity = slave_intensity[1:stride_line:end,1:stride_sample:end]
            
            # save 
            slave_path = joinpath(result_folder,string(index[2])*polarization*"_itens.jld")
            master_path = joinpath(result_folder,string(index[1])*polarization*"_itens.jld")
            coherence_path = joinpath(result_folder,string(index[1])*string(index[2])*
                                                            polarization*"_coher.jld")
            println("Save")
            JLD.save(slave_path, "data", slave_intensity)
            JLD.save(coherence_path, "data", complex_coherence)
            if i == 1
                println("Save master intens")
                master_intensity = master_intensity[1:stride_line:end,1:stride_sample:end]
                JLD.save(master_path, "data", master_intensity)
            end
            
        end
        
    end
    
    println("Save AUX info")
    lines = lines[1:stride_line:end]
    samples = samples[1:stride_sample:end]
    
    # save lines and samples 
    line_sample_path = joinpath(result_folder,"line_sample.jld")
    JLD.save(line_sample_path, "lines", lines, "samples", samples)
    
    # Save heights
    heights = Misc.interp_grid(lut1["master_line"] ,lut1["master_sample"],
        reshape(lut1["heights"],(length(lut1["master_line"]),length(lut1["master_sample"])))
        ,lines, samples);
    
    heights_path = joinpath(result_folder,"heights.jld")
    JLD.save(heights_path, "data", heights)
    
    return 1
    
end

preprocess3_v1 (generic function with 2 methods)

# Huston 

In [4]:
dem_path = "/Users/simon/Data/DEM/houston_small.jld";
dem_nan = -25;

co_folder = "/Users/simon/Data/Sentinel/houston/S1B_IW_SLC__1SDV_20170830T122203_20170830T122233_007169_00CA2C_C92C.SAFE"
pre1_folder = "/Users/simon/Data/Sentinel/houston/S1A_IW_SLC__1SDV_20170824T122248_20170824T122318_018065_01E54E_5C27.SAFE"
pre2_folder = "/Users/simon/Data/Sentinel/houston/S1B_IW_SLC__1SDV_20170818T122205_20170818T122235_006994_00C525_0F49.SAFE"
product_folders = [co_folder,pre1_folder,pre2_folder];

### River

In [5]:
result_folder = "/Users/simon/Documents/DTU fag/Speciale/results/houston/river_BAB"
master_B_view = [7600:9500,7000:15000];

preprocess3_v1(product_folders,master_B_view,dem_path,1, result_folder,dem_nan)

VV
load POD
load DEM
i: 1
coreg no lut
Save
Save master intens
i: 2
coreg no lut
Save
VH
i: 1
coreg with lut
Save
Save master intens
i: 2
coreg with lut
Save
Save AUX info


1

### city center

In [5]:
result_folder = "/Users/simon/Documents/DTU fag/Speciale/results/houston/city_center_BAB"
master_B_view = [7600:9500,11000:19000];

preprocess3_v1(product_folders,master_B_view,dem_path,1, result_folder,dem_nan)

VV
load POD
load DEM
i: 1
coreg no lut
Save
Save master intens
i: 2
coreg no lut
Save
VH
i: 1
coreg with lut
Save
Save master intens
i: 2
coreg with lut
Save


1

### Resovior

In [9]:
result_folder = "/Users/simon/Documents/DTU fag/Speciale/results/houston/resovoir_BAB"
master_B_view = [7000:9500,500:5000];

preprocess3_v1(product_folders,master_B_view,dem_path,2, result_folder,dem_nan)

VV
load POD
load DEM
i: 1
coreg no lut
Save
Save master intens
i: 2
coreg no lut
Save
VH
i: 1
coreg with lut
Save
Save master intens
i: 2
coreg with lut
Save


1

## Vejle

In [3]:
dem_path = "/Users/simon/Data/DEM/denmark.jld";
dem_nan = 40;

In [4]:


co_folder =   "/Users/simon/Data/Sentinel/vejle_2020/S1A_IW_SLC__1SDV_20200217T053252_20200217T053319_031288_03996E_8F3C.SAFE"
pre1_folder = "/Users/simon/Data/Sentinel/vejle_2020/S1B_IW_SLC__1SDV_20200211T053200_20200211T053227_020217_026479_5817.SAFE"
pre2_folder = "/Users/simon/Data/Sentinel/vejle_2020/S1A_IW_SLC__1SDV_20200205T053252_20200205T053319_031113_039360_7E5C.SAFE"

result_folder = "/Users/simon/Documents/DTU fag/Speciale/results/vejle_2020/factory_kolding"

master_A_view = [7800:9600,10000:20000];
product_folders = [co_folder,pre1_folder,pre2_folder];

preprocess3_v1(product_folders,master_A_view,dem_path,3, result_folder,dem_nan)

VV
load POD
load DEM
i: 1
coreg no lut
Save
Save master intens
i: 2
coreg no lut
Save
VH
i: 1
coreg with lut
Save
Save master intens
i: 2
coreg with lut
Save
Save AUX info


1

In [4]:
co_folder =   "/Users/simon/Data/Sentinel/vejle_2020/S1A_IW_SLC__1SDV_20200217T053252_20200217T053319_031288_03996E_8F3C.SAFE"
pre1_folder = "/Users/simon/Data/Sentinel/vejle_2020/S1B_IW_SLC__1SDV_20200211T053200_20200211T053227_020217_026479_5817.SAFE"
pre2_folder = "/Users/simon/Data/Sentinel/vejle_2020/S1A_IW_SLC__1SDV_20200205T053252_20200205T053319_031113_039360_7E5C.SAFE"

result_folder = "/Users/simon/Documents/DTU fag/Speciale/results/vejle_2020/factory"

master_A_view = [6200:7800,10000:20000];;
product_folders = [co_folder,pre1_folder,pre2_folder];

preprocess3_v1(product_folders,master_A_view,dem_path,3, result_folder,dem_nan)


VV
load POD
load DEM
i: 1
coreg no lut
Save
Save master intens
i: 2
coreg no lut
Save
VH
i: 1
coreg with lut
Save
Save master intens
i: 2
coreg with lut
Save
Save AUX info


1

In [11]:
co_folder = "/Volumes/VERBATIM HD/Data_sentinel/vejle_oktober/S1A_IW_SLC__1SDV_20191011T170932_20191011T170959_029414_035865_06AB.SAFE"
pre1_folder = "/Volumes/VERBATIM HD/Data_sentinel/vejle_oktober/S1A_IW_SLC__1SDV_20190929T170932_20190929T170959_029239_035263_C3F9.SAFE"
pre2_folder = "/Volumes/VERBATIM HD/Data_sentinel/vejle_oktober/S1A_IW_SLC__1SDV_20190917T170931_20190917T170958_029064_034C5E_2484.SAFE"
result_folder = "/Users/simon/Documents/DTU fag/Speciale/results/vejle_oktober_AAA"

master_A_view = [8000:10000,1200:6000];
product_folders = [co_folder,pre1_folder,pre2_folder];

preprocess3_v1(product_folders,master_A_view,dem_path,3, result_folder,dem_nan)

VV
load POD
load DEM
i: 1
coreg no lut
Save
Save master intens
i: 2
coreg no lut
Save
VH
i: 1
coreg with lut
Save
Save master intens
i: 2
coreg with lut
Save
Save AUX info


1

In [6]:
co_folder = "/Users/simon/Data/Sentinel/vejle_oktober/S1A_IW_SLC__1SDV_20191011T170932_20191011T170959_029414_035865_06AB.SAFE"
pre1_folder = "/Users/simon/Data/Sentinel/vejle_oktober/S1B_IW_SLC__1SDV_20191005T170849_20191005T170916_018343_0228DC_F565.SAFE"
pre2_folder = "/Users/simon/Data/Sentinel/vejle_oktober/S1A_IW_SLC__1SDV_20190929T170932_20190929T170959_029239_035263_C3F9.SAFE"

result_folder = "/Users/simon/Documents/DTU fag/Speciale/results/vejle_oktober_ABA"

product_folders = [co_folder,pre1_folder,pre2_folder];

master_B_view = [9500:11500,1200:6000];

preprocess3_v1(product_folders,master_B_view,dem_path,3, result_folder,dem_nan)

VV
load POD
load DEM
i: 1
coreg no lut
Save
Save master intens
i: 2
coreg no lut
Save
VH
i: 1
coreg with lut
Save
Save master intens
i: 2
coreg with lut
Save
Save AUX info


1

## Vejle / ribe 4

In [3]:
dem_path = "/Users/simon/Data/DEM/denmark.jld";
dem_nan = 40;

co_folder =   "/Users/simon/Data/Sentinel/south_jutland_jan/S1A_IW_SLC__1SDV_20200220T170928_20200220T170955_031339_039B3F_8FCB.SAFE"
pre1_folder = "/Users/simon/Data/Sentinel/south_jutland_jan/S1B_IW_SLC__1SDV_20200214T170846_20200214T170913_020268_026619_0A0D.SAFE"
pre2_folder = "/Users/simon/Data/Sentinel/south_jutland_jan/S1A_IW_SLC__1SDV_20200208T170928_20200208T170956_031164_03953C_E56D.SAFE"


"/Users/simon/Data/Sentinel/south_jutland_jan/S1A_IW_SLC__1SDV_20200208T170928_20200208T170956_031164_03953C_E56D.SAFE"

In [5]:


# Vejle 
result_folder = "/Users/simon/Documents/DTU fag/Speciale/results/south_jutland_jan/vejle_1"

master_A_view = [9000:11300,1000:10000];
product_folders = [co_folder,pre1_folder,pre2_folder];

preprocess3_v1(product_folders,master_A_view,dem_path,3, result_folder,dem_nan)

POE not found. Dowload initiated
No POE avalible, Get RES instead


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

POE not found. Dowload initiated


100  546k  100  546k    0     0   732k      0 --:--:-- --:--:-- --:--:--  732k


No POE avalible, Get RES instead


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

VV


100  546k  100  546k    0     0   814k      0 --:--:-- --:--:-- --:--:--  813k


load POD
load DEM
i: 1
coreg no lut
Save
Save master intens
i: 2
coreg no lut
Save
VH
i: 1
coreg with lut
Save
Save master intens
i: 2
coreg with lut
Save
Save AUX info


1

In [6]:
# Vejle 2
result_folder = "/Users/simon/Documents/DTU fag/Speciale/results/south_jutland_jan/vejle_2"

master_A_view = [10500:12800,16000:25300];
product_folders = [co_folder,pre1_folder,pre2_folder];

preprocess3_v1(product_folders,master_A_view,dem_path,2, result_folder,dem_nan)



VV
load POD
load DEM
i: 1
coreg no lut
Save
Save master intens
i: 2
coreg no lut
Save
VH
i: 1
coreg with lut
Save
Save master intens
i: 2
coreg with lut
Save
Save AUX info


1

In [4]:
# Ribe 
result_folder = "/Users/simon/Documents/DTU fag/Speciale/results/south_jutland_jan/ribe"

master_A_view = [6200:8000,4000:16000];
product_folders = [co_folder,pre1_folder,pre2_folder];

preprocess3_v1(product_folders,master_A_view,dem_path,2, result_folder,dem_nan)

VV
load POD
load DEM
i: 1
coreg no lut
Save
Save master intens
i: 2
coreg no lut
Save
VH
i: 1
coreg with lut
Save
Save master intens
i: 2
coreg with lut
Save
Save AUX info


1